##### GRUPO: 3
##### SECCIÓN: 3
##### INTEGRANTES: 
######                     Acevedo Rivero, Williams  (25%)    Función Búsqueda Local y Metaheuristico
######                             Leyton Reto, Eva  (25%)    Función Solución Inicial
######                             Paredes Herrera, Lisbeth  (25%)    Función Matriz Distancias, penalidad  y Vecindad1
######                             Zevallos Astahuaman, Deily   (25%)   Función Objetivo(Costo Total)
#####  Entrega: 2

# Packages

Los packages deben ser agregados desde el Terminal.

In [1]:
using CSV

# Nuestra Lógica

Para el desarrollo del proyecto, se utilizó el método Clarke and Wright. Así, se obtuvo los mayores ahorros entre pares de vdc's. Luego, cuando se tuvo agrupados en pares y ordenados de mayor a menor (de acuerdo al ahorro que producian), se fromaron rutas pares, todo ello, tomando en cuenta la restricción de que cada ruta no sobrepase la capacidad que tiene un camión.
Por otro lado, también se creó una función "penalidad", basándose en el tiempo que se toma recorrer cada ruta, la cual no debe ser mayor a 10 horas. Esta función lo que hace es aplicar una multa de 1 millón de soles, al costo de la ruta, por ende, al costo total. Con esto se puede asegurar que, cuando se aplique el metaheuristico, se mejore sí o sí, la solución inicial.
   
El costo total está calculado por la suma del costo de cada ruta que es la aplicación de la fórmula brindada más la penalidad y el costo extra, en caso exista.
   
 # Metaheuristico escogido
 
 
Para este proyecto hemos decidido trabajar con el método de **Tabu search**. Este es un método metaheurístico muy conocido, usado en problemas de optimización. Básicamente, el metahurístico Tabú Search (TS) está diseñado para obtener una solución muy cerca de un óptimo global, en un problema de optimización general. Por otro lado, este método se compone de varios componentes cuya combinación en proporciones adecuadas, hará de TS un procedimiento eficiente. 

La forma básica de Tabu Search (TS) se basa en ideas propuestas por Fred Glover (1977, 1986). El método se basa en procedimientos diseñados para cruzar los límites de viabilidad u optimalidad local, en lugar de tratarlos como
barreras. TS se basa en la premisa de que la resolución de problemas debe incorporar memoria adaptativa y exploración responsiva. La función de memoria adaptativa de TS permite la implementación de procedimientos que son capaces de buscar el espacio de solución económicamente y eficazmente factible, además, el uso de memoria se utiliza para no repetir la trayectoria de búsqueda. 
 
La lógica de nuestro proyecto, parte desde una solución aleatoria inicial. Por otro lado, en cada iteración, una nueva solución es tomada del vecindario (función vecindad), predefinido de la solución inicial (actual).  Seguidamente, el valor de la función objetivo de la nueva solución es comparado con la actual para determinar si se ha mejorado o empeorado la solución. 

Como en todo el proceso, para mejorar nuestra solución incial se aplicará Tabu search, con espacio de búsqueda y una lista Tabu en donde se guarden los intercambios, todo ello con el fin de mejorar nuestra solución. Además, este método nos permite aceptar soluciones que empeoran la solución actual, ya que si el intercambio se encuentra presente en la lista tabu, este no se tomará y en caso no se encuentre, tomará la solución asi esta empeore. 
En el tabu search siempre se trabaja con un criterio de aspiración ya que en cualquiera de los dos casos antes mencionados si la solución mejora, se guardará dicha solución asi esta se encuentre en la lista tabu.

Finalmente, una de las ventajas de usar este método es que nos permite localizar correctamente los VDC (nodos), lo cual reduce el espacio de búsqueda de rutas de la planta a un VDC y entre VDCs. Por otro lado, el procedimeinto de Tabu Search, no cosidera horas de cierre o descanso y solo está basado en las restricciones descritas a detalle en las descripción de las estructuras, funciones y función objetivo. 
 

# Funciones

### 1. Funcion matriz de distancias

In [2]:
#=Para la creación de la matriz distancia, primero creamos una matriz cuadrada de 0's 
    con las dimensiones de la cantidad de VDC's más la planta y le fuimos 
    agregando las distancias entre cada par de VDC's y tambien entre la planta =#
function matriz_distancia(archivo)
    matriz_distance=zeros(length(archivo[:,5]),length(archivo[:,6]))
    for i in 1:length(archivo[:,1]) ## corremos en los elementos del archivo tomando referencia la primera columna
        for j in 1:length(archivo[:,1])
            matriz_distance[i,j]=1.2*sqrt((archivo[i,5]-archivo[j,5])^2+(archivo[i,6]-archivo[j,6])^2)
                                    ##formula euclidiana para hallar la distancia entre dos puntos
        end
    end
    return matriz_distance 
end

matriz_distancia (generic function with 1 method)

### 2. Funcion objetivo: `costoTotal()`

In [3]:
#En este apartado, se encuentran detalladas cada uno de las funciones con las que trabajamos.
#=La función camiones_alquiler nos ayuda a encontrar los camiones que tenemos que alquilar; esta función se activa, siempre
y cuando la cantidad de rutas, sobrepase la cantidad de camiones propios.Cabe indicar que por cada ruta, corresponde un camión=#
function camiones_alquiler(archivo,ruts)
    q=0
    if length(ruts)>=archivo[1,1] 
        q=length(ruts)-archivo[1,1]
    end
    return q
end

#=Esta función compara la cantidad de rutas con la cantidad de camiones propios. 
Así, si la cantidad de rutas es menor o igual a la cantidad de camiones propios, los camiones propios a utilizar serán 
este valor=#
function camiones_propios_usar(archivo,ruts,camion_alquiler)
    return length(ruts)-camion_alquiler
end

#=Esta función se encargará de mostrar la cantidad de rutas que requerirán que se alquilen camiones.
Esta función se activará, siempre que la cantidad de camiones requeridos, supere la cantidad de camiones propios. 
Asimismo, será preferible que, como se paga la misma cantidad por camión alquilado,
(indistintamente de la cantidad de horas utlizadas) será preferible que las rutas que utilcen camiones alquilados,
tengan un tiempo de ruta menor igual muy cercano a 8 horas, para reducir los costos operacionales.=#
function rut_use_alqui(ruts,camion_alquiler,tipo_ruta)
    rutas_posibles=tipo_ruta[]
    for i in 1:length(ruts)
       if ruts[i].tiempo<=8 
            push!(rutas_posibles,ruts[i])
        end
    end
    sort!(rutas_posibles,lt=(x,y)->x.tiempo>y.tiempo)
    rutas_usadas=tipo_ruta[]
    if length(rutas_usadas)>camion_alquiler
        for i in 1:length(rutas_posibles)
            push!(rutas_usadas,rutas_posibles[i])
        end
    else
        for i in 1:camion_alquiler
            push!(rutas_usadas,rutas_posibles[i])
        end
    end
    return rutas_usadas
end

#=Esta función devuelve las rutas que han usado camiones propios. 
Los parámetros que se utilizan son la cantidad de rutas usadas, a las mismas que se les restará las rutas que usan camiones
alquilados=#
function rut_use_propios(rutaas,rutas_usadas_camion_alquiler,tipo_ruta)
    for i in 1:length(rutas_usadas_camion_alquiler)
       filter!(e->e!=rutas_usadas_camion_alquiler[i],rutaas)
    end
    return rutaas
end

#= Esta función devuelve los costos de las rutas propias. 
Además, muestra cuáles son las rutas que exceden nuestros parámetros de tiempo (mayor de 10 horas)=#
function costo_propio(rutas_usadas,tipo_ruta)
    acumulador=0
    for i in 1:length(rutas_usadas)
       aux=costo_ruta(rutas_usadas[i])
        acumulador+=aux
    end
    return acumulador
end

#=Esta función se encarga de calcular los costos de horas extra. 
Para esta función, se requiere analizar las rutas cuyas horas excenden al tiempo de 8 horas. 
Sabiendo, además, que solo son rutas que utilizan camiones propios. 
Por otro lado, el costo de horas extras son 33 soles por cada hora=#
function costo_extra(ruta)
    acumulador=0
       if ruta.tiempo>8
            aux=ceil(ruta.tiempo-8)
            acumulador+=33*aux
        end
    return acumulador
end

#=Esta función calcula el costo de cada una de las rutas. Para ello, hemos decidido trabajar en función del tiempo (horas).
Le hemos asignado: 65 soles (costo por hora de operación), 176 soles (costos de chofer por ruta),
penalidad (si se excede las 10 horas máximas perminitdas) y el costo extra de cada ruta=#
function costo_ruta(ruta)
    acumulador=(ruta.tiempo)*65+176+penalidad_distancia(ruta)+costo_extra(ruta)
    return acumulador
end

#=Esta función calcula el tiempo total de todas las rutas. Para ello, se le han asignado todos los parámetros y funciones 
descritas anteriormente. 
Así, esta función retorna el costo total en general, con las penalidades por cada ruta, en caso existan.=#
function costoTotal(ruts,archivo,tipo_ruta)
    x=deepcopy(ruts)
    camion_alquiler=camiones_alquiler(archivo,x)
    camion_usar_propios=camiones_propios_usar(archivo,x,camion_alquiler)
    rutas_usadas_camion_alquiler=rut_use_alqui(x,camion_alquiler,tipo_ruta)
    rutas_usadas_camion_propios=rut_use_propios(x, rutas_usadas_camion_alquiler,tipo_ruta)
    costo_alquiler=1392*camion_alquiler
    costo_prop=costo_propio(rutas_usadas_camion_propios,tipo_ruta)
    println("El costo total es: ")
    total=costo_alquiler+costo_prop
    println("costo de alquiler= $costo_alquiler  costo de camiones propios= $costo_prop")
    return total
end


costoTotal (generic function with 1 method)

### 3. Funcion de construccion: `solucionInicial()`

In [4]:
#=Primero, se decidió crear funciones que nos ayudan a trabajar el problema. Asimismo, se tendrá una solución inicial.
Esta solución se irá mejorando con los métodos que se apliquen, a medidad que se desarrolla el proyecto.=#

#=Como ya tenemos creada la matriz distancia, en la cual su triangulo superior e inferior son iguales, 
se usará el triangulo inferior para guardar los ahorros que obtenemos al juntar dos rutas, 
eso será posible con la siguiente matriz: =#

function matriz_ahorro(matriz_distances::Array)
    for i in 2:length(matriz_distances[:,1]) # i corre en los todos los elementos de la primera columna de la matriz distancia. 
        for j in i+1:length(matriz_distances[1,:])# i corre en los elemntos de la primera fila de matriz ditancia.
            matriz_distances[j,i]=matriz_distances[i-1,i]+matriz_distances[i-1,j]-matriz_distances[i,j] 
        end
    end
    for i in 1:length(matriz_distances[:,1]) 
        matriz_distances[i,1]=0.0
    end
    return matriz_distances
end

#=Esta función se encarga de generar nodos (VDC). Además, retornará cada posición del nodo y el orden de visita.=#
function generar_nodos(tipo_nodo,archivo)
    lista_nodos=tipo_nodo[]
    for i in 1:length(archivo[:,4])
        push!(lista_nodos,tipo_nodo(archivo[i,4],archivo[i,5],archivo[i,6],archivo[i,7]))
    end
    return lista_nodos
end

#= Se creó esta función para que, más adelante, nos ayude a invertir el orden del array.=#
function invierte!(arreglo)
    if length(arreglo)%2==0
        for i in 1:floor(Int,length(arreglo)/2)
            temporal=arreglo[i]
            arreglo[i]=arreglo[length(arreglo)+1-i]
            arreglo[length(arreglo)+1-i]=temporal
        end
    else
        for i in 1:floor(Int,(length(arreglo)+1)/2)
            temporal=arreglo[i]
            arreglo[i]=arreglo[length(arreglo)+1-i]
            arreglo[length(arreglo)+1-i]=temporal
        end
    end
    return arreglo
end

#=Esta función permite crear un array de ahorros.
Además, aplicando la función anterior (invierte!), se pudo ordenar de forma decreciente los ahorros 
(los mayores arriba y los menores abajos) =#
function array_ahorros(matriz_savings::Array)
    array_ahorros=Float64[]
    for i in 3:length(matriz_savings[:,1])
        for j in 2:i-1
            push!(array_ahorros,matriz_savings[i,j])
        end
    end
    sort!(array_ahorros)
    invierte!(array_ahorros)
    return array_ahorros
end

#= Se creó una función sin_planta, ya que la función array_nodos_ahorro te retorna una lista en orden de los pares de VDC's 
que merecen estar juntos incluyendo la planta, y esta función se encargará de eliminarla de esa lista=#
function sin_planta(tipo_nodo,array)
    array_nodo_def=tipo_nodo[]
    for i in 1:length(array)
        if array[i].ident != "Planta"
            push!(array_nodo_def,array[i])
        end
    end
    return array_nodo_def
end

#= Esta funcion me devolverá un array de los pares de vdc ordenados de acuerdo al ahorro que producen=#
function array_nodos_ahorro(tipo_nodo,array_ahorros,matriz_savings,lista_nodos)
   array_nodos=tipo_nodo[]
    for ñ in 1:length(array_ahorros)
        #este for indexado [i,j] recorrerá toda la matriz de ahorros para encontrar donde se encuentra el que estamos buscando
        for i in 1:length(matriz_savings[:,1])
            for j in 1:length(matriz_savings[1,:])
                if matriz_savings[i,j]==array_ahorros[ñ]
                    aux=0
                    for p in 1:length(array_nodos)
                        if lista_nodos[i]==array_nodos[p]||lista_nodos[j]==array_nodos[p]
                            aux+=1
                        end
                    end
                    if aux==0
                        if lista_nodos[i] != lista_nodos[j]
                            push!(array_nodos,lista_nodos[i])
                            push!(array_nodos,lista_nodos[j])
                        else
                            push!(array_nodos,lista_nodos[i])
                        end
                    end
                end
            end
        end
    end
    array_nodos_def=sin_planta(tipo_nodo,array_nodos)
    return array_nodos_def
end

#= Esta funcion me devolverá una solución inicial cuando la cantidad de vdc's 
sea par, ya que así se podrá trabajar con parejas de vd´s en su totalidad=#
function sol_ini_par(tipo_nodo,almacen,cap_camion,ruts,lista_nodos,tipo_ruta)
    while length(almacen)>0
        acumulador=tipo_nodo[]
        cambiable_almacen=tipo_nodo[]
        aux=0
        for j in 1:2:length(almacen)
            if aux<cap_camion
                if aux+almacen[j].Dem+almacen[j+1].Dem<=cap_camion
                    push!(acumulador,almacen[j])
                    push!(acumulador,almacen[j+1])
                    aux+=almacen[j].Dem+almacen[j+1].Dem
                else
                    push!(cambiable_almacen,almacen[j])
                    push!(cambiable_almacen,almacen[j+1])
                end
            else
                push!(cambiable_almacen,almacen[j])
                push!(cambiable_almacen,almacen[j+1])
            end
        end
        distancia=distancia_ruta(acumulador,lista_nodos)
        tiempo=ceil(distancia/70)+ceil(length(acumulador)*0.5)
        almacen=cambiable_almacen
        demanda=calc_demanda(acumulador)
        push!(ruts,tipo_ruta(lista_nodos[1],acumulador,lista_nodos[1],distancia,tiempo,demanda))
    end
    return ruts
end

#=La siguiente función se encarga de calcular la distancia entre cada nodo. 
Para ello, se aplicó la fórmula de distancia Euclidiana.=#
function dist(nodo1,nodo2)
    t=1.2*sqrt((nodo1.c_x-nodo2.c_x)^2+(nodo1.c_y-nodo2.c_y)^2)
    return t
end

#= ESta función sirve para acumular todas las distancias entre nodos en cada ruta.=#
function distancia_ruta(array,lista_nodos)
        acumulador=dist(lista_nodos[1],array[1])+dist(array[length(array)],lista_nodos[1])
        for i in 1:length(array)-1
            acumulador+=dist(array[i],array[i+1])
        end
    return acumulador
end

#Esta función nos permite calcular la cantidad de demanda de cada ruta
function calc_demanda(rut)
    demanda=0
   for i in 1:length(rut)
        demanda+=rut[i].Dem
    end
    return demanda
end

#=Si la cantidad de vdc's es impar se usará esta funcion para crear la solución inicial ya que buscará a que ruta
puede integrar el ultimo vdc, el cual es el que no tiene pareja 
de ahorro por que es un ahorro bajo o porque sus parejas ya habian sido tomadas por un mejor ahorro y como 
nosotros agregamos a las rutas de par en par, sobrara uno y por eso necesitamos dos funciones para la solución inicial=#
function sol_ini_impar(tipo_nodo,almacen,cap_camion,ruts,array_nodos_ahorros,lista_nodos,tipo_ruta)
    interruptor=0
    ultimo_nodo=array_nodos_ahorros[length(array_nodos_ahorros)]
    while length(almacen)-1>0
        acumulador=tipo_nodo[]
        cambiable_almacen=tipo_nodo[]
        aux=0
        for j in 1:2:length(almacen)-1
            if aux<cap_camion
                if aux+almacen[j].Dem+almacen[j+1].Dem<=cap_camion 
                    push!(acumulador,almacen[j])
                    push!(acumulador,almacen[j+1])
                    aux+=almacen[j].Dem+almacen[j+1].Dem
                else
                    push!(cambiable_almacen,almacen[j])
                    push!(cambiable_almacen,almacen[j+1])
                end
            else
                push!(cambiable_almacen,almacen[j])
                push!(cambiable_almacen,almacen[j+1])
            end
        end
        if interruptor==0
            if aux+ultimo_nodo.Dem<=cap_camion
                push!(acumulador,ultimo_nodo)
                interruptor+=1
            end
        end
        distancia=distancia_ruta(acumulador,lista_nodos)
        tiempo=ceil(distancia/70)+ceil(length(acumulador)*0.5)
        almacen=cambiable_almacen
        demanda=calc_demanda(acumulador)
        push!(ruts,tipo_ruta(lista_nodos[1],acumulador,lista_nodos[1],distancia,tiempo,demanda))
    end
    return ruts
end

#=Finalmente, se creó la solución incial utilizando todas las funciones anteriormente explicadas. 
Cabe indicar, nuevamente, que se trabajó todo en función del tiempo, expresado en horas.=#
function solucionInicial(tipo_ruta,array_nodos_ahorros,archivo,lista_nodos,tipo_nodo)
    ruts=tipo_ruta[]
    almacen=array_nodos_ahorros
    cap_camion=archivo[1,2]
    if length(array_nodos_ahorros)%2==0
        ruts=sol_ini_par(tipo_nodo,almacen,cap_camion,ruts,lista_nodos,tipo_ruta)
    else
        ruts=sol_ini_impar(tipo_nodo,almacen, cap_camion,ruts, array_nodos_ahorros, lista_nodos,tipo_ruta)
    end
    sort!(ruts,lt=(x,y)->x.tiempo>y.tiempo)
    return ruts
end

solucionInicial (generic function with 1 method)

### 4. Funcion de vecindades: `generarVecinos()`

In [5]:
#= Para poder encontrar una solución mejor a la ya obtenida, se decidió crear vecindades. 
Aquí, se pondrán obtener nuevas soluciones (mejores o peores) para nuestro problema.
Así, pues, se decidió tomar en cuenta las siguientes consideraciones:
- Tomar un VDC de una ruta e intercambiarlo con un VDC de otra ruta.
- Tomar un VDC de una ruta y colocarlo dentro de otra ruta, todo ello, sin sobrepasar la capacidad máxima restringida en cada camión=#

#= Primero, se decidió crear una función ruta_hecha, esta función calcula de forma correcta el tiempo y demanda de cada ruta, en el
en el total de rutas, para asi poder calcular y comparar de una manera correcta.=#
function ruta_hecha(tipo_ruta,nruta,rutai,lista_nodos,archivo)
    rutas_hecha=tipo_ruta[]
    interruptor=0
    for i in 1:length(nruta)
        x=nruta[i].vdc_jun
        distancia=distancia_ruta(x,lista_nodos)
        tiempo=ceil(distancia/70)+ceil(length(x)*0.5)
        demanda=calc_demanda(x)
        if demanda>archivo[1,2]
           interruptor+=1 
        end
        push!(rutas_hecha,tipo_ruta(lista_nodos[1],x,lista_nodos[1],distancia,tiempo,demanda))
    end  
    if interruptor>=1
       rutas_hecha=rutai 
    end
    sort!(rutas_hecha,lt=(x,y)->x.tiempo>y.tiempo)
    return rutas_hecha
end

#=Se creó una función vecindad1, la misma que, valiéndose de una copia de la solución inicial,
obtendrá dos VCD´s random, los mismos que procederá a intercambiar. 
Las consideraciones son que ambos VCD´s sean distintos y que ambos VDC´s pertenezcan a rutas diferentes=#
function vecindad1(rutai,archivo,lista_nodos,tipo_ruta)
    nruta=deepcopy(rutai)
    i=rand(1:length(nruta))
    j=rand(1:length(nruta))
    if i!=j
        k=rand(1:length(nruta[i].vdc_jun))
        l=rand(1:length(nruta[j].vdc_jun))
        
        x=nruta[i].vdc_jun[k]
        nruta[i].vdc_jun[k]=nruta[j].vdc_jun[l]
        nruta[j].vdc_jun[l]=x
    end
     original_ruta=ruta_hecha(tipo_ruta,nruta,rutai,lista_nodos,archivo)
    return original_ruta
end



vecindad1 (generic function with 1 method)

### 5. Funcion de busqueda: `busquedaLocal()`

In [6]:
#=La función de busquedaLocal lo que hace es buscar entre todas las vecindades obtenidas,
en un intervalo de 1 a 1500 iteraciones, la mejor solución.
Para ello, lo que hará será comparar cada uno de los costos obtenidos. 
Así, empezará con el costo de la solución incial como si fuera el mejor costo obtenido,
luego comparará cada costo y así, la solución que tenga un costo menor, será nuestra nueva solución.=#

function busquedaLocal(solInicial,archivo,tipo_ruta)
    solucion=deepcopy(solInicial)
    costo_solucion=costoTotal(solucion,archivo,tipo_ruta)
    for i in 1:1500
        aux=vecindad1(solucion,archivo,lista_nodos,tipo_ruta)
        costo_prueba=costoTotal(aux,archivo,tipo_ruta)
        if costo_prueba<costo_solucion
            solucion=aux
            costo_solucion=costo_prueba
        end
    end
    return solucion
end

busquedaLocal (generic function with 1 method)

### 6. Funcion metaheuristico: `metaheuristico()`

In [19]:
#=En cuanto a la función metahuristico, lo que esta función hace es, en un inicio, 
crear una lista tabu del tipo nodo vacía. Seguidamente, tomará la mejor solución obtenida en nuestra búsqueda local. 
Luego, hará una copia de la solución y obtendrá 1 ruta random y luego, en esa misma ruta, obtendrá dos VDC´s random.
Así pues, tendrá una lista tabu con sus VDC´s. =#

function metaheuristico(solInicial,archivo,lista_nodos,tipo_ruta,tipo_nodo)
    lista_tabu=tipo_nodo[]
    solucion=busquedaLocal(solInicial,archivo,tipo_ruta)
    nruta=deepcopy(solucion)
    n1=rand(1:length(nruta))
    n2=rand(1:length(nruta[n1].vdc_jun))
    n3=rand(1:length(nruta[n1].vdc_jun))
    push!(lista_tabu,nruta[n1].vdc_jun[n2])
    push!(lista_tabu,nruta[n1].vdc_jun[n3])
    
    #=Seguidamente, escogerá dos rutas random y un VDC random de cada una de las rutas. 
    Luego, estos VDC´s los intercambiará. Hará esto por 1500 iteraciones.=#
    for ñ in 1:1500
        i=rand(1:length(nruta))
        j=rand(1:length(nruta))
        k=rand(1:length(nruta[i].vdc_jun))
        l=rand(1:length(nruta[j].vdc_jun))
        if i !=j
            x=nruta[i].vdc_jun[k]
            nruta[i].vdc_jun[k]=nruta[j].vdc_jun[l]
            nruta[j].vdc_jun[l]=x
        end
        original_ruta=ruta_hecha(tipo_ruta,nruta,solucion,lista_nodos,archivo)
        if costoTotal(original_ruta,archivo,tipo_ruta)<costoTotal(solucion,archivo,tipo_ruta)
            solucion=original_ruta
            nruta=original_ruta
        end
        #=Luego, verificará la posicón de cada VDC, es decir, si es par o no.
        Si es par, se compara el VDC junto con el VDC anterior, ya que es entre pares de nodos, y la pareja del Vdc encontrado
        estaá en la posición anterior=#
        for h in 1:length(lista_tabu)
            if h%2==0
               if lista_tabu[h]!=original_ruta[i].vdc_jun[k] && lista_tabu[h-1]!=original_ruta[j].vdc_jun[l]
                    nruta=original_ruta 
                end
            end
            #=Si es impar, se guarda en la lista tabu el VDC más el VDC siguiente, viceversa a lo anterior=#
            if h%2==1
               if lista_tabu[h]!=original_ruta[i].vdc_jun[k] && lista_tabu[h+1] !=original_ruta[j].vdc_jun[l]
                    nruta=original_ruta
                end
            end
        end
        #=En esta parte tomamos un limite para el tamaño de la lista tabu que guarde relación con el número de nodos en cuestión=#
        if length(lista_tabu)<=(ceil(length(lista_nodos)/2))
            push!(lista_tabu,original_ruta[i].vdc_jun[k])
            push!(lista_tabu,original_ruta[j].vdc_jun[l])
        else
            push!(lista_tabu,original_ruta[i].vdc_jun[k])
            push!(lista_tabu,original_ruta[j].vdc_jun[l])
            deleteat!(lista_tabu,1)
            deleteat!(lista_tabu,2)
        end
    end
    return solucion
end


metaheuristico (generic function with 1 method)

### 7. Funciones de penalidad

In [8]:
#=Esta función se encargará de penalizar a aquellas rutas que sobrepasen 10 horas en todo su trayecto.
Se decidió poner una penalidad grande, para que así, al momento de contruir nuestras soluciones, 
las rutas que tengan esta penalidad no se tomen como soluciones=#

function penalidad_distancia(ruta)
    acumulador=0
    if ruta.tiempo>10
        acumulador=10000
    end
    return acumulador
end


penalidad_distancia (generic function with 1 method)

# MAIN

### 0. Importar CSV

In [9]:
archivo=CSV.read("GM-Instance(previo).csv")

,Cantidad_Camiones,Capacidad_Camion,Nodo,Identificador,X_Coordinadas,Y_Coordenadas,Demanda
,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰
1,10,4,1,Planta,70,87,0
2,missing,missing,2,VDC1,70,28,2
3,missing,missing,3,VDC2,18,93,1
4,missing,missing,4,VDC3,3,19,1
5,missing,missing,5,VDC4,70,97,2
6,missing,missing,6,VDC5,1,68,1
7,missing,missing,7,VDC6,60,87,2
8,missing,missing,8,VDC7,46,31,2
9,missing,missing,9,VDC8,81,17,2


### 1. Estructuras de datos

In [20]:
mutable struct nodo_1111
    ident::String
    c_x::Int64
    c_y::Int64
    Dem::Int64
end

mutable struct ruta_123
    inicio::nodo_1111
    vdc_jun::Array{nodo_1111}
    fin::nodo_1111
    distancia::Float64
    tiempo::Float64
    demanda::Int64
end


### 2. Utilizar  funciones

In [21]:
matriz_distances=matriz_distancia(archivo)
matriz_savings=matriz_ahorro(matriz_distances)
lista_nodos=generar_nodos(nodo_1111,archivo)
lista_ahorros=array_ahorros(matriz_savings)
array_nodos_ahorros=array_nodos_ahorro(nodo_1111,lista_ahorros,matriz_savings,lista_nodos)
ruts_iniciales=solucionInicial(ruta_123,array_nodos_ahorros,archivo,lista_nodos,nodo_1111)
costoInicial = costoTotal(ruts_iniciales,archivo,ruta_123)
solucion_nueva=busquedaLocal(ruts_iniciales,archivo,ruta_123)
metaheuristico_aplicado=metaheuristico(solucion_nueva,archivo,lista_nodos,ruta_123,nodo_1111)

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4230.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4230.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4295.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4360.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4295.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4165.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4230.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4165.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo d

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4230.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4230.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo d

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4165.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4165.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo d

costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4035.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3970.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4100.0
El costo total es: 
costo de alquiler= 4176  costo d

...

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo d

costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 

costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0
El costo total es: 
costo de alquiler= 4176  costo d

13-element Array{ruta_123,1}:
 ruta_123(nodo_1111("Planta", 70, 87, 0), nodo_1111[nodo_1111("VDC20", 87, 93, 2), nodo_1111("VDC10", 42, 57, 2), nodo_1111("VDC11", 63, 81, 2)], nodo_1111("Planta", 70, 87, 0), 250.7673731225744, 6.0, 4)                            
 ruta_123(nodo_1111("Planta", 70, 87, 0), nodo_1111[nodo_1111("VDC19", 20, 65, 2), nodo_1111("VDC26", 19, 88, 1), nodo_1111("VDC5", 1, 68, 1), nodo_1111("VDC2", 18, 93, 1)], nodo_1111("Planta", 70, 87, 0), 216.3750109383988, 6.0, 4)
 ruta_123(nodo_1111("Planta", 70, 87, 0), nodo_1111[nodo_1111("VDC16", 40, 93, 2), nodo_1111("VDC3", 3, 19, 1)], nodo_1111("Planta", 70, 87, 0), 252.7325972466185, 5.0, 3)                                                             
 ruta_123(nodo_1111("Planta", 70, 87, 0), nodo_1111[nodo_1111("VDC29", 6, 14, 2), nodo_1111("VDC1", 70, 28, 2)], nodo_1111("Planta", 70, 87, 0), 206.5827517518449, 4.0, 4)                                                             
 ruta_123(nodo_1111("Planta", 70, 87, 



### 3. Imprimir resultados / graficas

In [22]:
#Función Costo aplicado para la solución inicial 
costoTotal(ruts_iniciales,archivo,ruta_123)

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 4230.0


8406.0

In [23]:
#Función Costo aplicado para la solución que nos brinda la busqueda local
costoTotal(solucion_nueva,archivo,ruta_123)

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0


8081.0

In [24]:
#Función Costo aplicado para la solución que nos brinda el metaheuristico
costoTotal(metaheuristico_aplicado,archivo,ruta_123)

El costo total es: 
costo de alquiler= 4176  costo de camiones propios= 3905.0


8081.0




### 4. Interpretar resultados

#### Respecto a la solución obtenida:
Como podemos observar, la solución va mejorando de acuerdo a como le vamos aplicando las funciones realizadas.
También podemos observar que el resultado de la función objetivo es un poco elevado y esto se debe a que estamos trabajando con rutas excedentes al tiempo máximo y la penalidad está siendo aplicada, estamos trabajando en una función para solucionar este incoveniente para la siguiente entrega.